In [48]:
import pandas as pd
import numpy as np
from datetime import date

In [49]:
# constants
FILE_PATH = "cars.csv"

## Data

This dataset was retrieved from [kaggle](https://www.kaggle.com/datasets/juanmerinobermejo/us-sales-cars-dataset). It provides comprehensive information about used cars available for sale in the United States. The dataset contains the following key attributes:

- **Brand:** The brand or manufacturer of the car.
- **Model:** The specific model of the car.
- **Mileage:** The number of miles the car has been driven.
- **Year:** The manufacturing year of the car.
- **Status:** Indicates whether the car is new, used, or certified pre-owned.
- **Dealer:** Information about the dealer or seller offering the car.
- **Price:** The listed price of the car in USD.

In [50]:
df = pd.read_csv(FILE_PATH, encoding="utf-16")
df.head()

,Brand,Model,Year,Status,Mileage,Dealer,Price
0,Mazda,CX-5,2023,New,NaN,NaN,36703.0
1,Kia,Sportage,2023,New,NaN,Classic Kia,28990.0
2,Chevrolet,Camaro,2024,New,NaN,Classic Chevrolet Beaumont,41425.0
3,Ford,Bronco,2023,Used,1551.0,Mike Smith Chrysler Dodge Jeep RAM,58900.0
4,Acura,TLX,2021,Used,30384.0,Mike Smith Nissan,34499.0


## Preprocessing the Data

To clean the data we will:
1. data cleaning
2. data transformation
3. feature reduction 

In [51]:
row_count = len(df)
print(f"Total Rows: {len(df)}")
df.isnull().sum().map(lambda sum: f"{sum / row_count:.2%}")

Total Rows: 51793


Brand       0.00%
Model       0.00%
Year        0.00%
Status      0.00%
Mileage    55.63%
Dealer      0.20%
Price       2.22%
dtype: object

### Missing Values
The statistics above indicate the percentage of missing data under each column. Many of the rows are missing entries for mileage.

In [52]:
print(f"Rows with missing mileage: {df.Mileage.count()}")
print(f"Rows with new status and missing mileage: {df[(df.Status == 'New') & (df.Mileage.isna())].Status.count()}")

Rows with missing mileage: 22981
Rows with new status and missing mileage: 28811


Replace the mileage to `0` of rows with new status and missing mileage then drop the rest NAs.

In [53]:
df.loc[(df['Status'] == 'New') & (df['Mileage'].isna()), 'Mileage'] = 0
df = df.dropna()
print(f"Total Rows: {len(df)}")

Total Rows: 50540


### Remove duplicates

In [54]:
df = df.drop_duplicates()
print(f"Total Rows: {len(df)}")

Total Rows: 49536


### Data Transformation / Normalization
- we should scale certain features (like year to age)
- we should check for any typos or inconsistencies in text

In [55]:
df["Age"] = date.today().year - df.Year

In [56]:
df.Status = df.Status.apply(lambda status: status.lower())
for status in df.Status.unique():
    print(status, end=",\t")

new,	used,	certified,	

In [57]:
df.Brand = df.Brand.apply(lambda brand: brand.lower())
for brand in df.Brand.unique():
    print(brand, end=",\t")

kia,	chevrolet,	ford,	acura,	volkswagen,	gmc,	bmw,	hyundai,	jeep,	dodge,	infiniti,	honda,	lexus,	toyota,	cadillac,	buick,	mazda,	lincoln,	mercury,	mini,	ram,	land rover,	audi,	nissan,	mercedes,	porsche,	subaru,	jaguar,	maserati,	alfa romeo,	tesla,	bentley,	genesis,	volvo,	chrysler,	mitsubishi,	fiat,	rivian,	aston martin,	lamborghini,	hummer,	saturn,	rolls-royce,	ferrari,	polestar,	lucid,	scion,	pontiac,	karma,	smart,	mclaren,	lotus,	saab,	maybach,	suzuki,	international scout,	geo,	oldsmobile,	isuzu,	

In [58]:
df.Model = df.Model.apply(lambda model: model.lower())
for model in df.Model.unique():
    print(model, end=",\t")

sportage,	camaro,	bronco,	tlx,	golf,	yukon,	m340,	sonata,	grand cherokee,	wagoneer,	durango,	mustang,	f-350,	qx30,	passport,	es,	sierra,	tundra,	ct5,	envista,	cx-5,	civic,	accord,	atlas,	4runner,	sequoia,	continental,	marquis,	is,	lx,	rz 450e,	escalade,	ls,	nautilus,	integra,	mdx,	435 i,	ilx,	palisade,	x7,	escape,	cooper,	ix m60,	x3,	canyon,	land cruiser,	lyriq,	trax,	1500,	navigator,	range rover,	rs 5,	leaf s,	altima,	e-class,	gls 600,	highlander,	gls 450,	cayenne,	q7,	tt,	wrangler,	gt,	f-150,	f-250,	defender,	qx80,	qx60,	essentia,	gle,	panamera,	m3,	telluride,	forte,	a7,	x5,	330 i,	ix xdrive50,	740,	juke,	ct6,	mazda3,	wrx,	crosstrek,	ranger,	explorer,	town car,	fusion,	2500,	gladiator,	q5,	f-pace,	discovery,	silverado,	gla,	g 550,	e-tron,	s-class,	ghibli,	tahoe,	6,	cr-v,	elantra,	gx,	odyssey,	giulia,	corvette,	c-class,	cx-9,	cl,	x4,	3,	suburban,	fiesta,	tacoma,	model x,	flying spur,	roadster,	corsair,	mx-5 miata,	xd,	tucson,	rx,	lc,	macan,	taycan,	rdx,	outback,	gv80,	g70,	challenger,

In [59]:
df.describe()

,Year,Mileage,Price,Age
count,49536.000000,49536.000000,4.953600e+04,49536.000000
mean,2020.866481,24753.097121,5.126538e+04,2.133519
std,4.275406,40373.370957,3.580619e+04,4.275406
min,1959.000000,0.000000,1.990000e+03,-1.000000
25%,2020.000000,0.000000,3.079900e+04,0.000000
50%,2023.000000,0.000000,4.548000e+04,0.000000
75%,2023.000000,38564.500000,6.310500e+04,3.000000
max,2024.000000,400396.000000,1.299995e+06,64.000000
